# Import

In [66]:
import pandas as pd
import numpy as np

# from IPython.display import display
# pd.options.display.max_columns = 500
# pd.options.display.max_rows = 500

%matplotlib inline

In [67]:
train_raw = pd.read_csv('data/train.csv')

In [68]:
train_df = pd.read_csv('data/train.csv')
train_df = train_df.drop('Id', axis = 1)

In [69]:
test_df = pd.read_csv('data/test.csv')

In [70]:
y = train_df['SalePrice']
train_df = train_df.drop('SalePrice', axis=1)

# Divide varbles by type

In [71]:
# date variables
date_vars = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']

# list comprehension of continuous variables that are not dates
cont_vars = [cont_var for cont_var in train_df.columns if
        (train_df[cont_var].dtype == 'float64') | (train_df[cont_var].dtype == 'int64')
        and not cont_var in date_vars]

# change continus varibles to float
train_df[cont_vars] = train_df[cont_vars].astype('float64')

# create dataframe of date variables
dates = train_df[date_vars].astype(object)

In [72]:
# dict compehension of the frequncy of unique varibales
cont_freq = {cont_var: train_df[cont_var].nunique() for cont_var in cont_vars}
# frequency data frame
cont_freq_df = pd.DataFrame.from_dict(cont_freq, orient='index').rename(columns={0: 'freq'})
# continuous variables with few unique values
low_vol_cont = ['LowQualFinSF', '3SsnPorch', 'PoolArea']
# variabkes with Nan as the mode
mode_na = ['Alley', 'PoolQC', 'Fence','MiscFeature', 'FireplaceQu']


In [73]:
# data frame with numerical variables 
nums = train_df[list(cont_freq_df[cont_freq_df.freq >= 50].index)].astype(object)

num_cats = train_df[list(cont_freq_df[cont_freq_df.freq < 50].index)].astype(object)

# list of non categorical variables
non_cat = list(nums.columns) + list(dates.columns) + list(num_cats.columns) + mode_na
# data frame with only categorical
cats =  train_df.drop(non_cat, axis = 1)

# data frame with categoricals where the mode is nan
cats_na = train_df[mode_na]

# move continuous variables with low-frequency from num_cats to nums
nums[low_vol_cont] = num_cats[low_vol_cont]
num_cats = num_cats.drop(low_vol_cont, axis=1)

# Fill numarical missing with 0

In [74]:
nums = nums.fillna(0)

# Add dummies remove dominate and original categorical columns

In [75]:
def add_dummies_remove_modes(dummy_list, df_raw, df):
    """ 
    add_dummies_remove_modes (dummy_list, df_raw, df)
    takes a list of column names `dummy_list` to dummify then drop 
    after dummification, a reference dataframe `df_raw` to search for 
    the dominant value of each varable in `dummy_list` then drop the 
    dominant dummy variable and a dataframe `df` you wish to concat 
    dummified variables to.
    """

    dummy_modes = list(df_raw[dummy_list].mode().iloc[0,:].items())

    dummy_modes = [(col, (float(mode))) 
                   if type(mode) == int else (col, mode) for col, mode in dummy_modes]

    drop_modes = list(map(lambda x: str(x[0]) + '_' + str(x[1]), dummy_modes))

    dummy_cols = pd.get_dummies(
        df[dummy_list].astype(object), dummy_na=True).drop(drop_modes, axis=1)

    return pd.concat([
       df.drop(dummy_list, axis=1),
       dummy_cols], axis=1)


# Numarical Categorical to Dummies

In [76]:
num_cats_raw = num_cats

# # impute numarical categoricals with mode
# num_cats = num_cats.fillna(num_cats.mode())

# # dummify categotical numericals
# num_cats = add_dummies_remove_modes(list(num_cats.columns), train_df, num_cats)

# # drop columns that are all 0
# num_cats = num_cats.drop(
#     num_cats.columns[num_cats.sum()==0], axis=1)


# Categorical to Dummies

In [77]:
cats_raw = cats

#cats dummies
cats = add_dummies_remove_modes(list(cats.columns), train_df, cats)

cats = cats.drop(cats.columns[cats.sum()==0], axis=1)

# Dummify columns with NA as mode

In [78]:
cats_na_raw = cats_na
cats_na = pd.get_dummies(cats_na)

cats_na = cats_na.drop(cats_na.columns[cats_na.sum()==0], axis=1)

# Change GarageYrBlt to binary variable

In [79]:
dates['GarageYrBlt'] = dates['GarageYrBlt'].isna().apply(lambda x: int(not x))

In [80]:
test_df['GarageYrBlt'] = test_df['GarageYrBlt'].isna().apply(lambda x: int(not x))

# Concat final data frame

In [81]:
df_train_final = pd.concat([dates, nums, num_cats, cats, cats_na], axis=1)

# DF with raw featrues

In [82]:
df_train_raw = pd.concat([dates, nums, num_cats_raw, cats_raw.fillna(0), cats_na_raw.fillna(0)], axis=1)

In [83]:
df_test_raw = test_df

# Quality Check

In [84]:
df_train_final.shape

(1460, 261)

In [85]:
cols_before_drop = set(df_train_final.columns)
cols_after_drop = set(df_train_final.T.drop_duplicates(keep='first').T.columns)
cols_before_drop - cols_after_drop

{'BsmtCond_nan',
 'BsmtFinType1_nan',
 'Exterior2nd_CBlock',
 'GarageCond_nan',
 'GarageFinish_nan',
 'GarageQual_nan'}

In [86]:
cols_after_drop - cols_before_drop

set()

In [87]:
df_train_final = df_train_final.T.drop_duplicates(keep='first').T

In [88]:
df_train_final.shape

(1460, 255)

# Transform Dummies on Test

In [89]:
test_dummy_cols =  cats_raw.columns.tolist() +  cats_na_raw.columns.tolist()

In [90]:
 test_dummies1 = pd.get_dummies(test_df[test_dummy_cols])

In [91]:
test_df = pd.concat([test_dummies1, test_df], axis=1)

In [92]:
train_cols = set(df_train_final.columns)
test_cols = set(test_df.columns)

test_drop =  list(test_cols - train_cols)
test_unkonwn_dummies = list(train_cols - test_cols)

# Quality Check on Test

In [93]:
# these are all either the varibles that need to be droped after dummifyin 
# or are variables show up in train but not test
test_drop

['Street',
 'Alley',
 'BsmtFinType1_Unf',
 'LotConfig',
 'KitchenQual',
 'Condition2_Norm',
 'Exterior1st',
 'HeatingQC_Ex',
 'Functional_Typ',
 'PavedDrive',
 'Neighborhood_NAmes',
 'SaleCondition',
 'Id',
 'BsmtFinType2_Unf',
 'BsmtQual',
 'BsmtFinType1',
 'BsmtCond',
 'RoofMatl',
 'Foundation',
 'PoolQC',
 'PavedDrive_Y',
 'MasVnrType',
 'BsmtQual_TA',
 'LotShape_Reg',
 'BldgType_1Fam',
 'SaleType_WD',
 'CentralAir_Y',
 'CentralAir',
 'HouseStyle',
 'Exterior2nd_CBlock',
 'ExterCond_TA',
 'MasVnrType_None',
 'MiscFeature',
 'GarageType_Attchd',
 'BsmtExposure',
 'ExterCond',
 'GarageQual_TA',
 'BsmtExposure_No',
 'LandSlope',
 'LotConfig_Inside',
 'BldgType',
 'RoofStyle_Gable',
 'Utilities',
 'HouseStyle_1Story',
 'Electrical_SBrkr',
 'GarageFinish_Unf',
 'Fence',
 'GarageCond',
 'BsmtFinType2',
 'Exterior1st_VinylSd',
 'Utilities_AllPub',
 'GarageQual',
 'FireplaceQu',
 'SaleType',
 'LandSlope_Gtl',
 'Functional',
 'LotShape',
 'KitchenQual_TA',
 'HeatingQC',
 'Neighborhood',
 'MS

In [94]:
test_df.columns

Index(['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL',
       'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'LotShape_IR1',
       'LotShape_IR2', 'LotShape_IR3',
       ...
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal',
       'MoSold', 'YrSold', 'SaleType', 'SaleCondition'],
      dtype='object', length=314)

In [95]:
# these are the dummy variables that do not show up in test
test_unkonwn_dummies

['RoofMatl_Metal',
 'BsmtFinType2_nan',
 'Exterior1st_ImStucc',
 'BsmtQual_nan',
 'Heating_OthW',
 'Condition2_RRAn',
 'HouseStyle_2.5Fin',
 'Exterior2nd_Other',
 'Electrical_Mix',
 'Electrical_nan',
 'PoolQC_Fa',
 'Exterior1st_Stone',
 'Heating_Floor',
 'GarageType_nan',
 'MiscFeature_TenC',
 'BsmtExposure_nan',
 'GarageQual_Ex',
 'RoofMatl_Roll',
 'RoofMatl_Membran',
 'MasVnrType_nan',
 'RoofMatl_ClyTile',
 'Utilities_NoSeWa',
 'Condition2_RRAe',
 'Condition2_RRNn']

# Drop test columns that are not in train

In [96]:
test_df = test_df.drop(test_drop, axis=1)

# Add dummies that are unseen in test

In [97]:
test_dummies2 = test_df.reindex(columns = test_unkonwn_dummies, fill_value=0)

In [98]:
test_df = pd.concat([test_df, test_dummies2], axis=1)

# Impute missing with 0

In [99]:
test_na_cols = test_df.columns[test_df.isna().sum() > 0]
test_df[test_na_cols] = test_df[test_na_cols].fillna(0)

# Matching order of train and test

In [100]:
test_df = test_df[df_train_final.columns]

# Add response variable

In [101]:
df_train_final['SalePrice'] = train_raw['SalePrice']

In [102]:
df_train_raw['SalePrice'] = train_raw['SalePrice']

# Pickle

In [103]:
df_train_final.to_pickle('train.pkl')
test_df.to_pickle('test.pkl')

In [44]:
# df_train_raw.to_pickle('train_no_dummy.pkl')

In [38]:
# df_test_raw.to_pickle('test_no_dummy.pkl')